In [2]:
from src.InstrumentalVariable import InstrumentalVariable
from src.utils import *
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import cross_val_score

In [3]:
short_metrics_p, long_metrics_p = read_data(shift=True)
short_metrics = short_metrics_p[:, :, 0]
long_metrics = long_metrics_p[:, :, 0]

target_metric_p = long_metrics_p[:, 0, :]   # <--- here you can choose target (0, 1, 2, 3)
target_metric = target_metric_p[:, 0]

In [7]:
n_tests, n_metrics, _ = short_metrics_p.shape

inds = set(range(n_metrics))
inds.remove(14)

estimator = InstrumentalVariable(critical_p_value=95)

for n_basic_metrics in range(5):
    max_r2 = -10
    max_r2_feature = -1
    l_inds = list(inds)
    for j, feature_ind in enumerate(l_inds):
        target = short_metrics_p[:, feature_ind]
        target = apply_l0_regularization(target, 95)[:, 0]
        feature_inds = l_inds[:j] + l_inds[j+1:]
        features = short_metrics_p[:, feature_inds]
        r2 = cross_val_score(estimator, features, target, cv=5, scoring=make_scorer(r2_score), n_jobs=4)
        if np.mean(r2) > max_r2:
            max_r2 = np.mean(r2)
            max_r2_feature = feature_ind
        print(feature_ind, np.mean(r2))

    print(max_r2_feature, max_r2)
    inds.remove(max_r2_feature)

0 0.39620012728949694
1 0.8309941471545969
2 0.5329842480784732
3 0.9027843379254576
4 0.9146333082434503
5 0.673740099229774
6 0.7445092303444109
7 0.75653218270949
8 0.6722147707564631
9 0.8842997296265921
10 0.9054047552973936
11 0.6952898987473978
12 0.6950385045855038
13 0.7607974235733097
15 0.4078792559715838
16 0.7186806101060199
4 0.9146333082434503
0 0.14974658122249798


KeyboardInterrupt: 

In [5]:
n_tests, n_metrics, _ = short_metrics_p.shape

inds = set(range(n_metrics))
estimator = InstrumentalVariable(critical_p_value=95)

for i in range(n_metrics - 1):
    max_r2 = -10
    max_r2_feature = -1
    l_inds = list(inds)
    for j, feature_ind in enumerate(l_inds):
        target = short_metrics_p[:, feature_ind]
        target = apply_l0_regularization(target, 95)[:, 0]
        feature_inds = l_inds[:j] + l_inds[j+1:]
        features = short_metrics_p[:, feature_inds]
        r2 = cross_val_score(estimator, features, target, cv=5, scoring=make_scorer(r2_score), n_jobs=4)
        if np.mean(r2) > max_r2:
            max_r2 = np.mean(r2)
            max_r2_feature = feature_ind
        print(feature_ind, np.mean(r2))

    print(max_r2_feature, max_r2)
    inds.remove(max_r2_feature)    
    

0 0.3974991637599101
1 0.8295503820384255
2 0.5147481751138612
3 0.9343185117282262
4 0.9220428758944269
5 0.6665232563686068
6 0.984108670457519
7 0.7598626764267224
8 0.6702227599259387
9 0.8864960293911992
10 0.9073046090384818
11 0.6932813185980532
12 0.7274523901200566
13 0.7712799298436825
14 0.9901071807725046
15 0.8483777161172614
16 0.7121261337142921
14 0.9901071807725046
0 0.39620012728949694
1 0.8309941471545969


KeyboardInterrupt: 

In [8]:
def cor_selector(X, y):
    cor_list = []
    n_tests, n_metrics  = X.shape

    for i in range(n_metrics):
        cor = np.corrcoef(X[:, i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    return cor_list

regularized_short = np.array(list(map(lambda metric: apply_l0_regularization(metric, 95), short_metrics_p)))[:, :, 0]
regularized_target = apply_l0_regularization(target_metric_p, 95)[:, 0]
cor_feature = cor_selector(regularized_short, regularized_target)  

(296,) (296, 17)
17 selected features


In [15]:
print(np.argsort(np.abs(cor_feature)))
print(cor_feature)
arr = np.array(cor_feature)[np.argsort(np.abs(cor_feature))]

[ 8  1  6 14  2  7 11 16 15  0 10  9  4 13  3  5 12]
[0.2793248590078084, 0.0445019469448959, -0.061382713803989845, -0.427183423557067, -0.3366767135280396, -0.5253889317866387, 0.046300785472903394, 0.06275561727774981, 0.0045470962589972495, -0.33599727065274504, -0.30864891134071754, 0.11509941261083532, -0.5577178894198622, -0.382906299138516, -0.05050565807347423, 0.2194192157146221, 0.12675206238065906]


In [10]:
from scipy.special import softmax

In [17]:
print(softmax(arr))

[0.06464581 0.06728101 0.06740215 0.06118307 0.06052118 0.06852042
 0.07220256 0.07304884 0.08014162 0.08508928 0.04726298 0.04598793
 0.04595669 0.04388049 0.04197998 0.03805328 0.03684273]


In [16]:
print(arr)

[ 0.0045471   0.04450195  0.04630079 -0.05050566 -0.06138271  0.06275562
  0.11509941  0.12675206  0.21941922  0.27932486 -0.30864891 -0.33599727
 -0.33667671 -0.3829063  -0.42718342 -0.52538893 -0.55771789]


In [20]:
softmax(np.abs(cor_feature))

array([0.06093112, 0.0481789 , 0.0489991 , 0.07064042, 0.06452778,
       0.07792977, 0.04826564, 0.04906641, 0.04629186, 0.06448395,
       0.06274432, 0.05170314, 0.08049033, 0.0675809 , 0.04846902,
       0.05738818, 0.05230914])

In [21]:
np.abs(cor_feature)




array([0.27932486, 0.04450195, 0.06138271, 0.42718342, 0.33667671,
       0.52538893, 0.04630079, 0.06275562, 0.0045471 , 0.33599727,
       0.30864891, 0.11509941, 0.55771789, 0.3829063 , 0.05050566,
       0.21941922, 0.12675206])